# ExperimentRunner Demo: Bulk Experimentation

The `ExperimentRunner` enables systematic benchmarking across multiple parameters. Let's run a small batch experiment.

For transpiling to quantum hardware, we need to initialize backends first. Let's set up IBM Quantum access.

### How to Get IBM Credentials

To transpile to, or use real IBM Quantum hardware:

1. **Sign up** at [IBM Quantum Platform](https://quantum-computing.ibm.com/)
2. **Create an instance CRN** you currently need to setup a Pay-As-Yo-Go account at least
3. **Get your API token** directly from IBM Quantum Platform
4. **Choose a device** like `ibm_brisbane`, `ibm_torino`, etc.

In [ ]:
# Import the BackendManager for hardware backends
from sudoku_nisq.backends import BackendManager

# Initialize IBM Quantum backend
# Note: Replace with your actual IBM API token and instance
# For demo purposes, we'll show the initialization code but comment it out

# Modify these lines with your credentials:
api_token = "your_ibm_api_token_here"
instance = "crn:v1:bluemix:public:quantum-computing:us-east:a/your_instance_id"

BackendManager.authenticate_ibm(
    api_token=api_token,
    instance=instance
    )

IBM authentication successful
Found 6 IBM devices available to your account
Available devices: ['ibm_brisbane', 'ibm_fez', 'ibm_sherbrooke', 'ibm_torino', 'ibm_marrakesh', 'ibm_kingston']


['ibm_brisbane',
 'ibm_fez',
 'ibm_sherbrooke',
 'ibm_torino',
 'ibm_marrakesh',
 'ibm_kingston']

In [2]:
# Add backends from available devices

BackendManager.add_ibm_device(device="ibm_torino", alias="ibm_torino")
BackendManager.add_ibm_device(device="ibm_brisbane", alias="ibm_brisbane")

# Check registered backends
available_backends = BackendManager.aliases()
print(f"Registered backends: {available_backends}")

Registered backends: ['ibm_torino', 'ibm_brisbane']


You can also `BackendManager.init_ibm()` method handles both authentication and device registration in one step. Once initialized, you can access the backend anywhere in your code using `BackendManager.get("ibm_brisbane")`.

In [3]:
# Run a small batch experiment
# This will test ExactCoverQuantumSolver on the two backends, with 2 optimisation levels,
# with 2 encodings on 2 different puzzle difficulties, with 2 samples each

# Import solver class and experiment runner
from sudoku_nisq.exact_cover_solver import ExactCoverQuantumSolver
from sudoku_nisq.experiment_runner import ExperimentRunner

# Define solver configuration
solvers_config = {
    ExactCoverQuantumSolver: ["simple", "pattern"]
}

# Since we already authenticated backends above, we can just reference them by alias

# Create experiment runner with small parameters for demo
runner = ExperimentRunner(
    solvers=list(solvers_config.keys()),
    encodings_map=solvers_config,
    backends=["ibm_torino", "ibm_brisbane"],  # These must be registered in BackendManager
    opt_levels=[0, 1],
    subgrid_sizes=[2],  # Only 4x4 puzzles for demo
    num_missing_vals=[2],  # Two difficulty levels
    samples_per_combo=2,  # 2 samples per combination
    cache_base="./demo_cache",
    canonicalize=True,
    cache_transpiled=False
)

runner.run_batch()

Loaded seen index with 3 puzzle hashes
Starting experiment batch:
   • 1 solvers × 2 encodings
   • 2 backends × 2 optimization levels
   • 2 total puzzles
   • Cache: demo_cache

[1/2] Processing puzzle: 4×4, 2 missing, sample 1
Puzzle hash: 1d2d15a6...
ExactCoverQuantumSolver (simple)
Building circuit...
Circuit built in 0.05s
Transpiling ibm_torino (opt=0)...
Completed in 1.77s
Transpiling ibm_torino (opt=1)...
Completed in 1.53s
Transpiling ibm_brisbane (opt=0)...
Completed in 1.41s
Transpiling ibm_brisbane (opt=1)...
Completed in 2.54s
ExactCoverQuantumSolver (pattern)
Building circuit...
Circuit built in 0.05s
Transpiling ibm_torino (opt=0)...
Completed in 0.33s
Transpiling ibm_torino (opt=1)...
Completed in 1.18s
Transpiling ibm_brisbane (opt=0)...
Completed in 0.31s
Transpiling ibm_brisbane (opt=1)...
Completed in 2.96s
[2/2] Processing puzzle: 4×4, 2 missing, sample 2
Puzzle hash: 2e0d300c...
ExactCoverQuantumSolver (simple)
Building circuit...
Circuit built in 0.08s
Transpili